the upper part is the defination

In [1]:
import numpy as np

import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

#import tensorflow_addons as tfa

print(tf.__version__)


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#%%

folder = os.path.join('..','..','short_data_4_model','50')


data_train = np.load(os.path.join(folder, 'train_data.npz'),'r')
data_train = data_train['arr_0']

data_valid = np.load(os.path.join(folder, 'valid_data.npz'),'r')
data_valid = data_valid['arr_0']

print('data loaded successfully')

2.0.0
Num GPUs Available:  0
data loaded successfully


In [3]:
BATCH_SIZE = 60
seq_len =50
units = 200
# no use
embedding_dim = 256

num_examples = len(data_train)
steps_per_epoch = num_examples//BATCH_SIZE

dataset = tf.data.Dataset.from_tensor_slices(data_train).shuffle(num_examples)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [4]:
class Encoder(tf.keras.Model):
    def __init__(self, embedding_dim, encoding_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoding_units = encoding_units
        self.inputs = tf.keras.layers.Input((seq_len, 3))

        self.gru = keras.layers.GRU(self.encoding_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        inputs = x

        output, h = self.gru(inputs, initial_state = hidden)
        return inputs,output, h

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size,self.encoding_units))
    

example_input_batch = np.zeros((BATCH_SIZE,20,3))
# sample input
encoder = Encoder( embedding_dim, units, BATCH_SIZE)

sample_hidden = encoder.initialize_hidden_state()
embeded, sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)


print('embedding output shape: (batch size, sequence length, units) {}'.format(embeded.shape))
print('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

embedding output shape: (batch size, sequence length, units) (60, 20, 3)
Encoder output shape: (batch size, sequence length, units) (60, 20, 200)
Encoder Hidden state shape: (batch size, units) (60, 200)


In [5]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query: encoder hidden state  (batch size, units) 
        # values: encoder output  (batch size, sequence length, units)
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights
    
attention_layer = BahdanauAttention(units)

attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))



Attention result shape: (batch size, units) (60, 200)
Attention weights shape: (batch_size, sequence_length, 1) (60, 20, 1)


In [6]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, decoding_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.decoding_units = decoding_units
        #self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.decoding_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
        self.fc = keras.layers.Dense(vocab_size)
        
        self.fcn_4 = keras.layers.Dense(64, activation = 'relu')
        self.fcn_5 = keras.layers.Dense(32, activation='relu')

        # used for attention
        self.attention = BahdanauAttention(self.decoding_units)

    def call(self,  hidden, encoding_output,embedder):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, encoding_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        #x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), embedder], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        x = self.fc(output)


        return x, state, attention_weights
    

vocab_size=3
decoder = Decoder( vocab_size ,embedding_dim, units, BATCH_SIZE)

embeded = tf.expand_dims(embeded[:,0,:],1)
sample_decoder_output, _, _ = decoder(sample_hidden, sample_output, embeded)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))


Decoder output shape: (batch_size, vocab size) (60, 1, 3)


In [7]:
optimizer = keras.optimizers.Adam()
loss_object = tf.keras.losses.MeanSquaredError( reduction=tf.keras.losses.Reduction.NONE)

@tf.function
def loss_function(real, pred):
    
    #mask = tf.math.logical_not(tf.math.equal(real, [0,0,0]))
    loss_ = loss_object(real, pred)

    #mask = tf.cast(mask, dtype=loss_.dtype)
    #loss_ *= mask

    return tf.reduce_mean(loss_)

#checkpoint_dir = str(seq_len)+'test_seq2seq_Attenetion_'+str(units)+'_'+str(BATCH_SIZE)
checkpoint_dir = str(seq_len)+'full_seq2seq_Attenetion_'+str(units)+'_'+str(BATCH_SIZE)

if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)


print(checkpoint_dir)

50full_seq2seq_Attenetion_200_60


In [16]:
from plyfile import PlyData


from sklearn.metrics import mean_squared_error
from dipy.tracking.streamline import Streamlines
from dipy.tracking.streamlinespeed import length
                                          

class PlyStruct:
    """Class that process poly data"""

    def __init__(self):
        self.ply_data = None
        self.idx = None
        self.properties = None
        self.stream_line = None

    def load_poly_data(self, subpath: str, num_prop: int = None):
        """
        Load ply file
        :param str subpath: path of ply file
        :param int num_prop: optional parmeter to only take the first num properties (e.g. only x,y,z coordinates)
        """
        ply = PlyData.read(subpath)
        property_list = [ply['vertices'].data[name] for name in ply['vertices'].data.dtype.names]
        self.ply_data = np.array(property_list).T
        if num_prop:
            self.ply_data = self.ply_data[:, :num_prop]
        self.idx = ply['fiber'].data['endindex']
        self.properties = ply['vertices'].data.dtype.names
        if num_prop:
            self.properties = self.properties[:num_prop]
   

    def gen_stream_line(self):
        self.stream_line = []
        self.stream_line.append(self.ply_data[0:self.idx[0],:])
        for i in range(len(self.idx)-1):
            self.stream_line.append(self.ply_data[self.idx[i]:self.idx[i+1],:])


def zero_remove(darray):
    
    for i in range(np.shape(darray)[0]-1,-1,-1):
        if not (np.around(darray[i], decimals=0) == 0 ).all():
            return darray[0:i+1] 
            break


def get_length(stream):
    # to get the length of streams in this subject
    len_table = []
    for i in range(len(stream)):
        len_table.append(np.shape(stream[i])[0])
    return len_table

def cal_mse(data_raw,data_pred):
    mse = np.zeros(np.shape(data_raw)[0])
    for i in range(np.shape(data_raw)[0]):
        result = mean_squared_error(data_raw[i], data_pred[i])
        mse[i] = result
    return mse  


def ply2np(name):
    # convert a ply format to the matrix we are using
    # input : '128127_ex_cc-body_shore.ply', name of a oly file
    # optput: a matrix (#_of_fibers, #_of_vertex, 3 )
    temo = PlyStruct()
    temo.load_poly_data(os.path.join('..','data',name),num_prop=3)
    #data = temo.ply_data
  
    temo.gen_stream_line()
    stream = temo.stream_line


    stream = np.array(stream)
    len_table = get_length(stream)
    subject = np.zeros((len(stream),np.max(len_table),3))
    for i in range(len(stream)):
        length = len_table[i]
        subject[i,0:length,:] = stream[i]
        
    return subject

from sklearn.metrics import mean_squared_error
def cal_mse(data_raw,data_pred):
    mse = np.zeros(np.shape(data_raw)[0])
    for i in range(np.shape(data_raw)[0]):
        result = mean_squared_error(data_raw[i], data_pred[i])
        mse[i] = result
    return mse  

In [19]:
def evaluate(inputs):
    result = np.zeros((np.shape(inputs)))
    inputs = tf.convert_to_tensor(inputs)
  
    inference_batch_size = inputs.shape[0]
    hidden = [tf.zeros((inference_batch_size, units))]
    embeded_or,encoding_out, encoding_hidden = encoder(inputs, hidden)

    decoding_hidden = encoding_hidden



    for t in range(seq_len):
        embeded = tf.expand_dims(embeded_or[:,t,:],1)
        predictions, decoding_hidden, attention_weights = decoder(decoding_hidden, encoding_out, embeded)

    
        result[:,t,:] = tf.squeeze(predictions)

        # the predicted ID is fed back into the model
        decoding_input = predictions

    return result, inputs

# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()
'''
def make_pre(input):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))
'''

optimizer = keras.optimizers.Adam()
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# A way to calculate the validate error
'''
for i in range(20):
    f = open(os.path.join(checkpoint_dir,'checkpoint'), 'w')
    f.write('model_checkpoint_path: "ckpt-'+str(i+1)+'"\n')
    f.write('all_model_checkpoint_paths: "ckpt-'+str(i+1)+'"\n')
    f.close()
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    
    result, _ = evaluate(data_valid)
    mse_idv =loss_function(data_valid, result)
    print(str(i+1)+': '+str(mse_idv.numpy()))
'''

# type in the parameters
name : the file name\
len_thre: length control value, length below it will be considered abnomal fiber\
thre_con: control value of threshold percentile


In [15]:
name = '156031_ex_cc-body_shore.ply'
len_thre = 40
thre_con = 85

In [23]:
bundle = ply2np(name)

from dipy.tracking.streamlinespeed import (compress_streamlines, length,
                                           set_number_of_points)
streamlines_evl = Streamlines()
for j in range(np.shape(bundle)[0]):
    tmp = bundle[j]
    tmp = zero_remove(tmp)
    streamlines_evl.append(tmp)


subsamp_sls = set_number_of_points(streamlines_evl, seq_len)

data_test= np.array(subsamp_sls)
result, _ = evaluate(data_test)
mse_idv = cal_mse(data_test, result)


In [25]:
thre = np.percentile(mse_idv,thre_con)
pred_1 = 1*(mse_idv<thre)


bundle_str = Streamlines()

for i in range(np.shape(bundle)[0]):
    tmp = bundle[i]
    tmp = zero_remove(tmp)
    #tmp = tmp[~np.all(tmp == 0, axis=-1)]
    #tmp = np.around(tmp, decimals=0)
    bundle_str.append(tmp)
    
lengths = length(bundle_str)

pred_2 = 1*(lengths > len_thre)

pred = 1*((pred_1 + pred_2)>1)

op_path = 'result'
if not os.path.exists(op_path):
    os.mkdir(op_path)

save_path = os.path.join(op_path,'Detection'+name.split('_')[0]+'_Attention')
np.save(save_path,pred)